In [124]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import re

In [125]:
clean_data = pd.read_excel("/Users/Justin/GIT/data-question-2/violations2017.xlsx")

In [126]:
clean_data.head(20)

,VIOLATION_ID,ORG_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT,TABLE_NAME,EXPIRED_FLAG,DATE_EXPIRED,CREATED_BY,DATE_CREATED,MODIFIED_BY,DATE_MODIFIED,FEE_SETUP_ID,Unnamed: 14
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
1,2,1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
2,4,1,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
3,5,1,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
4,6,1,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
5,7,1,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
6,8,1,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
7,9,1,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
8,10,1,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
9,11,1,CERTOCC,Certificate of Occupancy Required,Section 16.24.130 – Certificate of Occupancy R...,CAAH_CERT_OCC,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN


In [127]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 15 columns):
VIOLATION_ID      928 non-null object
ORG_ID            789 non-null object
VIOLATION_TYPE    788 non-null object
VIOLATION_DESC    925 non-null object
VIOLATION_TEXT    666 non-null object
REMEDIAL_TEXT     345 non-null object
TABLE_NAME        137 non-null datetime64[ns]
EXPIRED_FLAG      651 non-null object
DATE_EXPIRED      2 non-null object
CREATED_BY        650 non-null float64
DATE_CREATED      651 non-null object
MODIFIED_BY       23 non-null object
DATE_MODIFIED     22 non-null datetime64[ns]
FEE_SETUP_ID      0 non-null float64
Unnamed: 14       0 non-null float64
dtypes: datetime64[ns](2), float64(3), object(10)
memory usage: 109.5+ KB


In [128]:
firstrows = clean_data[:300]

In [129]:
display(firstrows)

,VIOLATION_ID,ORG_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT,TABLE_NAME,EXPIRED_FLAG,DATE_EXPIRED,CREATED_BY,DATE_CREATED,MODIFIED_BY,DATE_MODIFIED,FEE_SETUP_ID,Unnamed: 14
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
1,2,1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
2,4,1,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
3,5,1,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
4,6,1,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
5,7,1,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
6,8,1,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
7,9,1,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
8,10,1,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
9,11,1,CERTOCC,Certificate of Occupancy Required,Section 16.24.130 – Certificate of Occupancy R...,CAAH_CERT_OCC,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN


In [130]:
#Moving the misentered data from row 15 up to the correct columns in row 14

firstrows.iloc[14,4] = str(firstrows.iloc[14,4]) + str(firstrows.iloc[15,0])
firstrows.iloc[14,5] = firstrows.iloc[15,1]
firstrows.iloc[14,6] = firstrows.iloc[15,2]
firstrows.iloc[14,7] = firstrows.iloc[15,3]
firstrows.iloc[14,8] = firstrows.iloc[15,4]
firstrows.iloc[14,9] = firstrows.iloc[15,5]
firstrows.iloc[14,10] = firstrows.iloc[15,6]
firstrows.iloc[14,11] = firstrows.iloc[15,7]
firstrows.iloc[14,12] = firstrows.iloc[15,8]
firstrows.iloc[14,13] = firstrows.iloc[15,9]



/Users/Justin/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [131]:
#removing row that was moved up to it's correct place and reset the index to be consecutive numbers

firstrows = firstrows.drop(firstrows.index[15])

firstrows = firstrows.reset_index(drop=True)
aust_vi_count = austin_data['DESCRIPTION'].value_counts()
firstrows.head(20)

,VIOLATION_ID,ORG_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT,TABLE_NAME,EXPIRED_FLAG,DATE_EXPIRED,CREATED_BY,DATE_CREATED,MODIFIED_BY,DATE_MODIFIED,FEE_SETUP_ID,Unnamed: 14
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
1,2,1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
2,4,1,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
3,5,1,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
4,6,1,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
5,7,1,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
6,8,1,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
7,9,1,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
8,10,1,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN
9,11,1,CERTOCC,Certificate of Occupancy Required,Section 16.24.130 – Certificate of Occupancy R...,CAAH_CERT_OCC,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN


In [132]:
aust_vi_count = austin_data['DESCRIPTION'].value_counts()

In [175]:
##attempting to create a loop to move info up one line and to the correct column

##for i, violation_id in enumerate('VIOLATION_ID'):
    ##if firstrows['vcount'].notnull:
        ##print(firstrows.vcount)
        #a = str(firstrows['VIOLATION_TEXT']) + str(firstrows['VIOLATION_ID'])
        #firstrows['VIOLATION_TEXT'] = a
        
##print(firstrows.head(20))

In [182]:
## Counting the number of times a word is repeated
text = clean_data['VIOLATION_DESC'].astype(str)
text = text.replace(',', ' ')
text = text.replace('(', ' ')
text = text.replace(')', ' ')
text = text.replace('/', ' ')
text = text.replace('-', ' ')
text = text.str.lower()

nash_vi_count = text.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)

In [183]:
## Sorting word counts in descending order

nash_vi_count = nash_vi_count.sort_values(ascending = False)

In [184]:
nash_vi_count = nash_vi_count.reset_index()
nash_vi_count.columns = ['word','count']

## Counting the word length to eliminate common words or symbols
nash_vi_count['length'] = nash_vi_count.word.str.len()
nash_bool = nash_vi_count['length'] > 3
nash_count = nash_vi_count[nash_bool == True]
nash_count.reset_index()

,index,word,count,length
0,2,exterior,111.0,8
1,6,area,46.0,4
2,7,prop,44.0,4
3,8,bldgs,44.0,5
4,11,water,36.0,5
5,12,permit,34.0,6
6,14,required,32.0,8
7,15,buildings,31.0,9
8,17,prohibited,27.0,10
9,18,occupancy,26.0,9


In [141]:
##Austin Data Set

austin_data = pd.read_csv('/Users/Justin/Documents/Violation_data/Austin_Violations.csv')

In [142]:
aust_vi_count = austin_data['DESCRIPTION'].value_counts()

aust_vi_count.head(20)

Property Abatement                  36626
Land Use Violation(s)               17522
Structure Condition Violation(s)    14261
Work Without Permit                  4052
SWS Chp 15 Violation(s)               149
Structure Condition Violations          7
Unknown                                 5
                                        3
Land Use Violations                     1
Name: DESCRIPTION, dtype: int64

In [143]:
##Chatanooga Data Set

chat_data = pd.read_csv('/Users/Justin/Documents/Violation_data/Chat_Violations.csv')

chat_data.head(50)

,Case Number,311 Case Number,Date Entered,Date Closed,Violation Description,Status,Street Number,Street Name,City,State,Latitude,Longitude,Location
0,CE20170008186,NaN,09/26/2017,10/03/2017,Overgrowth,Closed,1017,OAK ST,CHATTANOOGA,TN,35.042782,-85.290188,"(35.042781712000078, -85.290188488999945)"
1,CE20160004349,16-00048915,05/18/2016,06/06/2016,Litter,Closed,303,ASBURY DR,CHATTANOOGA,TN,35.010841,-85.233394,"(35.010841391, -85.23339419)"
2,CE20170007823,NaN,09/15/2017,10/03/2017,Litter,Closed,2108,E 18TH ST,CHATTANOOGA,TN,35.022265,-85.278551,"(35.022265373, -85.278550825999957)"
3,CE20160000800,16-00009692,02/10/2016,02/25/2016,Vehicle - Private Property,Closed,0,CALHOUN AVE,Chattanooga,TN,NaN,NaN,NaN
4,CE20150009076,15-00106051,09/02/2015,10/19/2015,Overgrowth - Vacant Lot,Closed,0,CORDELIA LN,CHATTANOOGA,TN,NaN,NaN,NaN
5,CE20160003356,16-00037845,04/25/2016,05/02/2016,Overgrowth - Vacant Lot,Closed,0,SHARP ST,Chattanooga,TN,NaN,NaN,NaN
6,CE20160007631,16-00089498,08/25/2016,08/29/2016,Overgrowth,Closed,1605,STARBOARD DR,HIXSON,TN,35.123635,-85.208011,"(35.123635274, -85.20801124)"
7,CE20170007516,NaN,08/31/2017,10/03/2017,Overgrowth - Vacant Lot,Closed,2405,KIRBY AVE,CHATTANOOGA,TN,35.029127,-85.269798,"(35.029127069, -85.269797951999976)"
8,CE20170004083,NaN,06/09/2017,06/23/2017,Overgrowth,Closed,3203,7TH AVE,CHATTANOOGA,TN,35.005596,-85.282679,"(35.005595855, -85.282679494999968)"
9,CE20160004254,16-00048023,05/17/2016,06/06/2016,Overgrowth - Vacant Lot,Closed,2502,BAILEY AVE,CHATTANOOGA,TN,35.030131,-85.267351,"(35.030130812, -85.26735122)"


In [144]:
chat_vi_count = chat_data['Violation Description'].value_counts()

chat_vi_count.head(50)

Overgrowth                    47506
Litter                        43617
Housing                       24426
Vehicle - Private Property    18844
Overgrowth - Vacant Lot       10903
Vehicle - On Street            5759
Dumping                        1435
Brush and Trash                 191
Garbage Set-Out                 165
Emergency Demo                    4
Name: Violation Description, dtype: int64

In [145]:
##Hartford Data Set

hart_data = pd.read_csv('/Users/Justin/Documents/Violation_data/Hartford_Violations.csv')

hart_data.head()

,Case Number,Parcel ID,Location,Location Description,Title,Complaint Violation,Open Date,Closed Date,Global Entity Name,First Name,Last Name,Inspector First Name,Inspector Last Name,geom
0,20638,153181001,191 WOODLAND DR,NaN,Housing Code Inspector,Certificate of Apartment Occupancy,04/11/2011,NaN,ST FRANCIS HOSPITAL OF HTFD,NaN,NaN,Tonja,Nelson,"(41.778834958931, -72.7014114629223)"
1,20767,169014054,665 BLUE HILLS AV,NaN,Public Health Inspector,Housing Code Enforcement,04/15/2011,04/18/2011,GETHSEMANE CHURCH OF GOD IN CHRIST INC,NaN,NaN,Dan,Lusa,"(41.8054656592273, -72.6957818013674)"
2,20621,250580229,38 BENTON ST,NaN,Public Health Inspector,Housing Code Enforcement,04/08/2011,NaN,SANCHEZ JOSE,NaN,NaN,Dan,Lusa,"(41.7475761674453, -72.6754478154558)"
3,20676,219217018,204 MAGNOLIA ST,NaN,Housing Code Inspector,Housing Code Enforcement,04/12/2011,NaN,88 NELSON REALTY LLC,NaN,NaN,Tonja,Nelson,"(41.7828839791892, -72.6864010685163)"
4,20763,164614240,587 BROADVIEW TER,NaN,Public Health Inspector,Rodent (HCE),04/18/2011,04/15/2011,ORTIZ IRIS C,NaN,NaN,Dan,Lusa,"(41.7362744629431, -72.7013272474807)"


In [146]:
hart_vi_count = hart_data['Complaint Violation'].value_counts()

hart_vi_count.head(50)

Housing Code Enforcement              12242
Certificate of Apartment Occupancy     3988
Essential Services                     2130
Bed Bugs (HCE)                         1301
Rodent (HCE)                            706
Rooming House / Hotel                   265
Vacate (HCE)                            184
Placards                                117
Fair Rent                                86
Emergency After Hours                    26
CAO Preventative                          3
Pest                                      2
Name: Complaint Violation, dtype: int64

In [147]:
##Kansas City Data Set

kc_data = pd.read_csv('/Users/Justin/Documents/Violation_data/KC_Violations.csv')

kc_data.head()

/Users/Justin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Property Violation ID,Case ID,Status,Case Opened Date,Case Closed Date,Days Open,Violation Code,Violation Description,Ordinance Number,Ordinance Chapter,...,State,Zip Code,Latitude,Longitude,KIVA PIN,Council District,Police Patrol Area,Inspection Area,Neighborhood,Code Violation Location
0,1082590,2013007106,Closed,01/18/2013,04/04/2013,67.0,NSEXTWAL03,ROTTING BOARDS ON EXTERIOR WALL,56-132 (A) C.O.,56,...,MO,64110.0,39.05223,-94.57551,133587,4.0,Central,78,South Hyde Park,"4109 CHARLOTTE ST\nMO 64110\n(39.05223, -94.57..."
1,1167308,2016121615,Closed,09/29/2016,06/05/2017,249.0,NSWLOPSTOR,UNAPPROVED STORAGE,48-32 C.O.,48,...,MO,64129.0,39.03082,-94.49209,51403,3.0,Metro,43,Eastwood Hills East,"5114 SYCAMORE AVE\nMO 64129\n(39.03082, -94.49..."
2,1121792,2014114280,Closed,09/03/2014,04/14/2015,223.0,NSTREE04,HAZARDOUS TREE BRANCHES,48-29 C.O.,48,...,MO,64134.0,38.91244,-94.50213,73684,6.0,South,173,Kirkside,"11614 MANCHESTER AVE\nMO 64134\n(38.91244, -94..."
3,1147988,2015127648,Closed,10/23/2015,NaN,452.0,NSWLVEH01,UNAPPROVED PARKING,48-28 C.O.,48,...,MO,64117.0,39.16647,-94.52383,76262,1.0,Shoal Creek,198,Chouteau Estates,"4701 NE PARVIN RD\nMO 64117\n(39.16647, -94.52..."
4,1091658,2013072945,Closed,05/31/2013,06/05/2014,272.0,NSWLWEED,RANK WEEDS OR UNATTENDED GROWTH,48-30 C.O.,48,...,MO,64114.0,38.92443,-94.59227,155254,6.0,South,148,Red Bridge South,"11145 WORNALL RD\nMO 64114\n(38.92443, -94.59227)"


In [148]:
kc_vi_count = kc_data['Violation Description'].value_counts()

kc_vi_count.head(50)

LITTER, TRASH, REFUSE, AND RUBBISH          82111
RANK WEEDS OR UNATTENDED GROWTH             68887
UNAPPROVED STORAGE                          61859
LIMBS AND BRUSH                             31769
PEELING, CRACKED, BLISTERED PAINT           26466
FAILURE TO REGISTER VACANT PROPERTY         24289
FAILURE TO REGISTER RENTAL DWELLING         19661
UNLICENSED VEHICLE                          17767
EXTERIOR SURFACE MATERIAL NOT PROTECTED     16150
UNAPPROVED PARKING                          14784
RANK WEEDS                                  13404
PAINTING NEEDED ON EXTERIOR WALL TRIM       13047
GENERAL DISREPAIR (GUTTER)                   8205
PANES ARE CRACKED, BROKEN OR MISSING         6730
ALL FENCES AND RET. WALLS KEPT IN REPAIR     6485
ANY BUILDING OR STRUCTURE OPEN TO ENTRY      6198
EXTERIOR SIDING MISSING                      5801
LOOSE, ROTTED, OR MISSING FASCIA BOARDS      5784
WRECKED, DAMAGED, OR DISABLED VEHICLE        5527
ACCUMULATION OF TRASH IN RIGHT OF WAY        5425


In [149]:
##Los Angeles Data Set

la_data = pd.read_csv('/Users/Justin/Documents/Violation_data/LA_Violations.csv')

la_data.head()

,Case Number,LADBS Inspection District,Address House Number,Address House Fraction Number,Address Street Direction,Address Street Name,Address Street Suffix,Address Street Suffix Direction,Address Zip,Date Case Generated,Date Case Closed,Parcel Identification Number (PIN),Case Type,Area Planning Commission (APC),Status of Case,Latitude/Longitude
0,143586,1114,1316.0,NaN,W,65TH,ST,NaN,90044.0,05/25/2005,01/10/2013,105B197 1500,GENERAL,South Los Angeles,O,"(33.98052, -118.2987)"
1,195751,2303,3932.0,NaN,S,BROADWAY,NaN,NaN,90037.0,03/11/2007,07/23/2016,115-5A203 85,VEIP,South Los Angeles,O,"(34.0115, -118.27815)"
2,233,2304,1855.0,NaN,W,GAGE,AVE,NaN,90047.0,10/27/2014,NaN,105B193 922,VEIP,South Los Angeles,O,"(33.98197, -118.3108)"
3,262090,8513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/28/2012,NaN,135A229 144,BILLBOARDS,East Los Angeles,O,NaN
4,264349,8513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/28/2012,NaN,138A235 6,BILLBOARDS,East Los Angeles,O,NaN


In [150]:
la_vi_count = la_data['Case Type'].value_counts()

la_vi_count.head(50)

GENERAL       8713
VEIP          1039
PACE           744
CNAP           371
CITATIONS      254
SIGNS          162
BILLBOARDS     127
LEA             65
FRP             13
XXX              1
Name: Case Type, dtype: int64

In [151]:
##New Orleans Data Set

no_data = pd.read_csv('/Users/Justin/Documents/Violation_data/NO_Violations.csv')

no_data.head(20)

,CaseID,ViolationID,CaseNo,Location,ViolationDate,CodeSection,Violation,Description,LastUpload
0,171833,547584,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:52:44 AM,28-11,28-11 Structure,Structural members shall be maintained structu...,02/18/2014 10:43:37 PM
1,171833,547585,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:53:03 AM,28-11,28-11 Floor Joists,Structural members including floor joists must...,02/18/2014 10:43:37 PM
2,171833,547590,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:53:55 AM,28-11,28-11 Studs,Structural members including studs must be mai...,02/18/2014 10:43:37 PM
3,171833,547592,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:54:20 AM,28-11,28-11 Rafters,Structural members including rafters must be m...,02/18/2014 10:43:37 PM
4,171833,547586,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:53:15 AM,28-13,28-13 Weatherboards,Exterior walls and weatherboards must be free ...,02/18/2014 10:43:37 PM
5,171833,547587,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:53:24 AM,28-13,28-13 Siding,Exterior walls and siding must be free from ho...,02/18/2014 10:43:37 PM
6,171833,547591,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:54:07 AM,28-13,28-13 Parapet Wall,"Exterior walls must be free from holes, breaks...",02/18/2014 10:43:37 PM
7,171833,547593,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:54:27 AM,28-14,28-14 Roof,"The roof and flashing must be sound, tight, an...",02/18/2014 10:43:37 PM
8,171833,547600,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:57:50 AM,28-15,28-15 Boarded and Secured,Unoccupied structures must be boarded and secu...,02/18/2014 10:43:37 PM
9,171833,547595,09-002855,6218-6220 Wainwright Dr,03/06/2013 10:55:28 AM,28-20,28-20 Rodent Harborage,All structures and exterior property must be k...,02/18/2014 10:43:37 PM


In [152]:
no_vi_count = no_data['CodeSection'].value_counts()

no_vi_count.head(100)

26-160(a)         14854
26-157            13896
26-161(a)         12336
28-33              8829
26-169             7856
26-172             7413
26-159             7320
26-167(b)          7261
26-171             6002
26-167(a)          5323
26-179             4923
28-11              4515
28-20              4413
28-25              3982
26-174             3806
28-14              3578
26-163             3397
28-13              3112
28-10              3108
26-181             2692
26-167(c)          2498
26-168             2372
28-29              2352
28-15              1959
26-176             1853
26-164             1602
26-175             1394
28-26               970
26-165              956
26-178              912
                  ...  
26-158              430
26-184              410
28-12               376
26-183              332
28-28               294
28-22               289
26-166              264
28-21(a)            227
26-180              188
26-201              115
28-23           

In [153]:
##New Orleans Data Set

sea_data = pd.read_csv('/Users/Justin/Documents/Violation_data/Seattle_Violations.csv')

sea_data.head(20)

,Case Number,Case Type,Address,Description,Case Group,Date Case Created,Last Inspection Date,Last Inspection Result,Status,Permit and Complaint Status URL,Latitude,Longitude,Location
0,1039286,TENANT RELOCATION ORDINANCE,1724 11TH AVE,Issuance of Owner's Certification of No Displa...,TENANT RELOCATION ASSIST ORD,01/20/2017,NaN,NaN,ADMINISTRATIVE CLOSURE,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.617182,-122.317827,"1724 11TH AVE\n(47.61718246, -122.31782678)"
1,1039274,TENANT RELOCATION ORDINANCE,2027 S WASHINGTON ST,Issuance of Certification of No Displacement.,TENANT RELOCATION ASSIST ORD,01/19/2017,NaN,NaN,ADMINISTRATIVE CLOSURE,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.600637,-122.305150,"2027 S WASHINGTON ST\n(47.60063701, -122.30515..."
2,1039267,HOUSING / ZONING,4840 CALIFORNIA AVE SW,1/17/17 rf Observed subject property to have e...,HOUSING,01/18/2017,01/17/2017,FAILED,OPEN,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.558019,-122.386528,"4840 CALIFORNIA AVE SW\n(47.55801855, -122.386..."
3,1039263,HOUSING / ZONING,1414 S COLUMBIAN WAY,01/18/2017 ED- Several HBMC violation. Smoke d...,HOUSING,01/18/2017,01/18/2017,FAILED,OPEN,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.571994,-122.314297,"1414 S COLUMBIAN WAY\n(47.57199355, -122.31429..."
4,1039254,HOUSING / ZONING,2410 E LYNN ST,"1/13/17- Observed no heat in the unit, temp. i...",HOUSING,01/13/2017,01/18/2017,PASSED,VOLUNTARY COMPLIANCE,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.639828,-122.301827,"2410 E LYNN ST\n(47.63982828, -122.30182736)"
5,1039244,CONSTRUCTION,3515 S FERDINAND ST,Permit 6222536 received final approval on 10/1...,BUILDING,01/12/2017,10/18/2016,PASSED,VOLUNTARY COMPLIANCE,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.557698,-122.288156,"3515 S FERDINAND ST\n(47.55769818, -122.28815605)"
6,1039243,CONSTRUCTION,2752 6TH AVE S,Permit 6454705 was issued 7/26/16 and received...,BUILDING,01/12/2017,08/23/2016,PASSED,VOLUNTARY COMPLIANCE,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.578336,-122.325642,"2752 6TH AVE S\n(47.57833626, -122.32564203)"
7,1039238,TENANT RELOCATION ORDINANCE,804 N MOTOR PL,Application for a Tenant Relocation License. T...,TENANT RELOCATION ASSIST ORD,01/11/2017,NaN,NaN,OPEN,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.658818,-122.348311,"804 N MOTOR PL\n(47.65881773, -122.34831133)"
8,1039228,TENANT RELOCATION ORDINANCE,2573 13TH AVE W,Issuance of Owner's Certification of Occupancy...,TENANT RELOCATION ASSIST ORD,01/10/2017,NaN,NaN,ADMINISTRATIVE CLOSURE,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.642803,-122.373902,"2573 13TH AVE W\n(47.64280333, -122.37390237)"
9,1039226,TENANT RELOCATION ORDINANCE,4426 4TH AVE NE,Issuance of Owner's Certification of Occupancy...,TENANT RELOCATION ASSIST ORD,01/09/2017,NaN,NaN,ADMINISTRATIVE CLOSURE,http://web6.seattle.gov/dpd/PermitStatus/Proje...,47.661050,-122.323737,"4426 4TH AVE NE\n(47.66105047, -122.32373743)"


In [154]:
sea_vi_count = sea_data['Case Group'].value_counts()

sea_vi_count.head(100)

ZONING                          14643
WEEDS AND VEGETATION             7141
BUILDING                         4691
HOUSING                          4398
VACANT BUILDING                  3073
SITE                             1506
JUST CAUSE EVICTION               874
TENANT RELOCATION ASSIST ORD      418
SIGNS                             355
CONDO/COOP CONVERSION             310
ELECTRICAL                        236
BUILDING AND PREMISES             227
CONSTRUCTION NOISE                197
MECHANICAL                        105
OTHER PROHIBITED ACTS             100
NONCONSTRUCTION NOISE              56
RENT RAISE/RRIO STDS NOT MET       24
TRAO AVOIDANCE                     14
OTHER CONSTRUCTION                 11
PRESALE                             8
PREMISES                            5
Name: Case Group, dtype: int64

In [155]:
##Virginia Beach Data Set

vb_data = pd.read_csv('/Users/Justin/Documents/Violation_data/VB_Violations.csv')

vb_data.head(20)

,Street Address,Zip Code,Sub-Division,Inspection Type,Case Type,Violation,Open Date,Closing Date,Location1
0,3117 FERRY FARM LN,23452-6528,PRINCESS ANNE PLAZA,Patrol,PROPERTY MAINTENANCE,SEC. 23-50b WEEDS AND GRASS,08/23/2016,08/24/2016,"3117 FERRY FARM LN\nVIRGINIA BEACH, VA 23452-6..."
1,4007 FRANCIS LEE DR,23452-1916,HOLLAND MEADOWS,Complaint,BUILDING_MAINT,604.3 ELECTRICAL SYSTEM HAZARDS,02/12/2016,02/18/2016,"4007 FRANCIS LEE DR\nVIRGINIA BEACH, VA 23452-..."
2,2020 ROUND HILL DR,23456,MASTER CARDS,Patrol,PROPERTY MAINTENANCE,SEC. 23-50b WEEDS AND GRASS,06/17/2016,09/16/2016,"2020 ROUND HILL DR\nVIRGINIA BEACH, VA 23456\n..."
3,5400 STONEHAVEN DR,23464-7014,CHARLESTOWNE,Patrol,BUILDING_MAINT,304.7 GUTTERS,02/09/2016,03/07/2016,"5400 STONEHAVEN DR\nVIRGINIA BEACH, VA 23464-7..."
4,6025 EDGELAKE DR,23464-4905,LEVEL GREEN TH THE VILLAGES,Patrol,INOPERABLE VEHICLES,16-40 INOPERABLE VEHICLE,03/15/2017,05/01/2017,"6025 EDGELAKE DR\nVIRGINIA BEACH, VA 23464-490..."
5,4425 DELRAY DR,23455-4533,THOROUGHGOOD/BLACKTHORNE,Complaint,RED_TAG,604.3 ELECTRICAL SYSTEM HAZARDS,12/27/2016,02/03/2017,"4425 DELRAY DR\nVIRGINIA BEACH, VA 23455-4533\..."
6,137 WAVERLY DR,23452-4325,PRINCESS ANNE PLAZA,Patrol,PROPERTY MAINTENANCE,"SEC. 23-50a TRASH, GARBAGE, LITTER",06/27/2017,07/06/2017,"137 WAVERLY DR\nVIRGINIA BEACH, VA 23452-4325\..."
7,1512 INDEPENDENCE BLVD,23455-4215,FENTRESS TOWN,Patrol,PROPERTY MAINTENANCE,"SEC. 23-50a TRASH, GARBAGE, LITTER",03/10/2016,03/15/2016,"1512 INDEPENDENCE BLVD\nVIRGINIA BEACH, VA 234..."
8,3536 RAINTREE RD,23452-4204,PRINCESS ANNE PLAZA,Patrol,PROPERTY MAINTENANCE,SEC. 23-50b WEEDS AND GRASS,07/27/2016,08/03/2016,"3536 RAINTREE RD\nVIRGINIA BEACH, VA 23452-420..."
9,3128 ASHLAWN TER,23452-6510,PRINCESS ANNE PLAZA,Patrol,INOPERABLE VEHICLES,16-41.2 PARKING AND STORAGE OF RECREATIONAL EQ...,02/13/2017,03/06/2017,"3128 ASHLAWN TER\nVIRGINIA BEACH, VA 23452-651..."


In [156]:
vb_vi_count = vb_data['Case Type'].value_counts()

vb_vi_count.head(100)

PROPERTY MAINTENANCE    30849
BUILDING_MAINT          14418
INOPERABLE VEHICLES      8548
WASTE_MANAGEMENT         2967
RED_TAG                  1663
Name: Case Type, dtype: int64

In [ ]:
violations = {}